In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
from helper_functions_from_TSMA import *
import pysam

path_to_bam_file = "/media/hieunguyen/GSHD_HN01/raw_data/bam_files/WGShg19.bam"
path_to_all_fa = "/media/hieunguyen/GSHD_HN01/storage/resources/hg19"
path_to_bed_file = "./methyl_regions/TSMA.bed"
bedfile = pd.read_csv(path_to_bed_file, sep="\t", header=None)
bedfile.columns = ["chrom", "start", "end", "region_name"]
bedfile["region"] = bedfile[["chrom", "start", "end"]].apply(lambda x: "{}:{}-{}".format(x[0], x[1], x[2]), axis=1)
readdf = fetch_reads(path_to_bam_file, "chr1:1000000-2000000")

region_chrom = "1"
region_start = 1000000
region_end = 2000000
refseq_at_cluster = get_refseq(path_to_all_fa = path_to_all_fa, 
                                chrom = region_chrom, 
                                start = region_start, 
                                end = region_end + 1)
all_cpg_in_cluster = [m.start(0) for m in re.finditer("CG", refseq_at_cluster)]
cpg_coords = [item + region_start for item in all_cpg_in_cluster]
cpg_coords_minus1 = [item -1 for item in cpg_coords]

cpg_coords = [str(item) for item in cpg_coords]
cpg_coords_minus1 = [str(item) for item in cpg_coords_minus1]

def assign_status_read_start(x, cpg_coords = cpg_coords, cpg_coords_minus1 = cpg_coords_minus1):
    if x in cpg_coords:
        return "CGN"
    elif x in cpg_coords_minus1:
        return "NCG"
    else:
        return "NA"
readdf["check_CG_motif"] = readdf["start"].apply(lambda x: assign_status_read_start(x))

/tmp/ipykernel_1973002/2996881333.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bedfile["region"] = bedfile[["chrom", "start", "end"]].apply(lambda x: "{}:{}-{}".format(x[0], x[1], x[2]), axis=1)


In [13]:
readdf.start.unique()

array(['1000705', '1000738', '1000831', ..., '1999404', '1999472',
       '1999581'], dtype=object)

In [10]:
readdf.groupby("check_CG_motif")["start"].count()

check_CG_motif
NA    8170
Name: start, dtype: int64